In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../")

# Huggingface dataset 불러오기

In [3]:
from pipeline.dataset.news import get_news_dataset

dataset = get_news_dataset()

/venv/final_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'bucket', 'text'],
    num_rows: 27400
})

In [4]:
print(dataset[-1]['text'])

스페셜리포트 원전 확대와 재생·수소 에너지 조화…尹 정부 실용적 에너지정책 전면에
새정부 에너지정책 방향은 에너지 분야 최상위 기본계획을 대체하는 만큼 윤석열 정부 에너지정책 방향을 집대성했다. 윤석열 대통령이 후보 시절 공약으로 제시한 원전 최강국 건설 과 국정과제에서 제시한 원전산업 생태계 강화 에너지안보 확립을 정책 곳곳에 반영했다. 정부는 공급 측면에서 원전을 중심으로 한 합리적인 에너지믹스 수요 관리 측면에서 시장원리에 기반한 에너지 수요 효율화 를 동시에 추진하겠다고 했다. 윤석열 정부의 국정철학을 반영하면서도 실용성까지 고민한 에너지정책을 제시했다. 5일 국무회의에서 의결된 새정부 에너지정책 방향 은 원전 비중 확대 등 에너지·탄소중립 관련 국정과제를 이행하기 위한 에너지정책 목표를 담았다. 윤석열 대통령직인수위원회는 국정과제에서 △탈원전 정책 폐기 원자력 산업 생태계 강화 △에너지안보 확립과 에너지 신산업·신시장 창출 △과학적인 탄소중립 이행방안 마련으로 녹색경제 전환을 국정과제로 제시한 바 있다. 정부는 국정과제 철학을 수행할 에너지정책을 마련하기 위해 산업계·학계·시민단체 등을 대상으로 충분한 의견수렴을 거쳤고 이전 정부 에너지정책을 대체하기 위한 새정부 에너지정책 방향 을 마련했다. 정부는 새정부 에너지정책 방향 에서 원전 확대를 중심으로 재생에너지와 석탄 LNG 등 발전원을 포함해 실현가능하고 합리적인 에너지 믹스를 재정립한다. 원전은 2030년 발전 비중을 30% 이상으로 확대하겠다는 구체적인 목표를 제시했다. 다른 에너지원은 발전원별 특성을 살려 적절한 발전 믹스를 도출할 계획이다. 구체적으로 재생에너지는 문재인 정부처럼 보급 속도전을 벌이지는 않고 태양광과 풍력 등 세부 발전원별 적정 비중도 다시 도출한다. 문재인 정부에서 대폭 감축할 계획이었던 석탄발전도 전력 수급상황을 고려해 다시 들여다본다. 특정 에너지원의 과도한 속도전은 지양하되 합리성에 기반한 발전 믹스를 다시 설계할 계획이다. 박일준 산업부 제2차관은 “지난 정부에서 에너

In [20]:
dataset

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'bucket', 'text'],
    num_rows: 27400
})

In [36]:
from sentence_transformers import SentenceTransformer
import torch

# Download from the 🤗 Hub
model = SentenceTransformer("dragonkue/bge-m3-ko")
print(f"Model is on device: {model.device}")

# Run inference
sentences = [ dataset[i]['text'] for i in range(64) ]
embeddings = model.encode(
    sentences,
    batch_size=64,
)
print(embeddings.shape)


Model is on device: cuda:0
(64, 1024)


In [5]:
from sentence_transformers import SentenceTransformer
import torch

# Download from the 🤗 Hub
model = SentenceTransformer("dragonkue/bge-m3-ko")

In [4]:
from pipeline.qdrant.client import QdrantService, models

qdrant = QdrantService()


In [5]:
qdrant.delete_collection(collection_name="bge_500_150")

In [14]:
from pipeline.retriever.embed import organize_chunks_for_upsert


In [45]:
points = organize_chunks_for_upsert(dataset, model, chunk_size=500, chunk_overlap=150, batch_size=16, index_from=2, index_to=15)

In [46]:
len(points)

57

In [47]:
points[2]

{'id': '2-2',
 'vector': [-0.0026812630239874125,
  -0.021672682836651802,
  -0.03556087240576744,
  0.034144334495067596,
  -0.0013473760336637497,
  -0.059219133108854294,
  -0.049087051302194595,
  -0.008382606320083141,
  -0.005014869384467602,
  0.004132579546421766,
  0.03681971877813339,
  0.04018082097172737,
  -0.00934824999421835,
  0.03191956877708435,
  0.021764034405350685,
  0.028585439547896385,
  0.010490359738469124,
  -0.03725508973002434,
  -0.05552993714809418,
  -0.01443520188331604,
  0.03214530646800995,
  0.01123639103025198,
  -0.021266531199216843,
  0.0042480588890612125,
  0.01667555794119835,
  -0.035057928413152695,
  0.05021218955516815,
  -0.03565702959895134,
  -0.013803195208311081,
  -0.04467805102467537,
  0.012052620761096478,
  0.005570349749177694,
  -0.02629614621400833,
  -0.012753847986459732,
  -0.020914651453495026,
  -0.025080380961298943,
  -0.025716006755828857,
  -0.030507516115903854,
  -0.07361830025911331,
  0.016018034890294075,
  0.0